In [1]:
from ortools.sat.cp_model_pb2 import CpSolverStatus
from pulp import PULP_CBC_CMD
from tqdm.notebook import tqdm, trange
import uuid

In [2]:
from loguru import logger as log

In [3]:
import numpy as np
from src.vrp_study.data_model import Tariff, Cargo, Node
from src.vrp_study.data_model import TariffCost
from src.vrp_study.pdptw_model.pdptw_routing_manager_builder import PDRoutingManagerBuilder
from src.vrp_study.configs import ModelConfig

from src.vrp_study.pdptw_model.routing_model import find_optimal_paths
from typing import Optional
from dataclasses import dataclass

In [4]:
from ortools.sat.python.cp_model import CpModel, CpSolver
from ortools.sat import sat_parameters_pb2

# from pyscipopt import Model, quicksum
# import pyscipopt
import pulp

# Подготовка данных

In [5]:
benchmark_type = 'pdp_100'
name = 'lc101.txt'

In [6]:
tariff = None
cargos: list[Cargo] = []
depo: Optional[Node] = None

In [7]:
id2info = {}
p2coordinates = {}
with open(f'../data/Li & Lim benchmark/{benchmark_type}/{name}', 'r') as file:
    for i, line in enumerate(file):
        line = line.split('\t')
        if i == 0:
            tariff = Tariff(
                id='car',
                capacity=int(line[1]),
                max_count=int(line[0]),
                cost_per_distance=[TariffCost(
                    min_dst_km=0,
                    max_dst_km=10000,
                    cost_per_km=1,
                    fixed_cost=0
                )]
            )
        else:
            c_id = int(line[0])
            x = int(line[1])
            y = int(line[2])

            mass = int(line[3])

            et = int(line[4])
            lt = int(line[5])
            st = int(line[6])

            pick_up = int(line[7])
            delivery = int(line[8])
            if pick_up == delivery:
                # print(12)
                depo = Node(
                    id=0,
                    cargo_id=c_id,
                    capacity=0,
                    service_time=0,
                    start_time=0,
                    end_time=lt,
                    coordinates=(x, y)
                )
                continue
            if pick_up == 0:
                if c_id not in id2info:
                    id2info[c_id] = {}
                id2info[c_id][0] = (x, y, mass, et, lt, st, c_id, delivery)
            else:
                delivery = c_id
                c_id = pick_up
                if c_id not in id2info:
                    id2info[c_id] = {}
                id2info[c_id][1] = (x, y, mass, et, lt, st, pick_up, delivery)


In [8]:
depo

Node(id=0, cargo_id=0, capacity=0, start_time=0, end_time=1236, service_time=0, coordinates=(40, 50))

In [9]:

for k, v in id2info.items():
    cargos.append(
        Cargo(
            id=k,
            nodes=[
                Node(
                    cargo_id=k,
                    id=v[i][6] if i == 0 else v[i][7],
                    capacity=v[i][2],
                    service_time=v[i][5],
                    start_time=v[i][3],
                    end_time=v[i][4],
                    coordinates=(v[i][0], v[i][1])
                )
                for i in range(2)
            ]
        )
    )

In [10]:
# cargos = cargos[:10]

In [11]:
p2coordinates.update({
    crg.nodes[i].id: crg.nodes[i].coordinates for crg in cargos for i in range(2)
})
p2coordinates[depo.id] = depo.coordinates
distance_matrix = {(u, v): np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2) for u, du in
                   p2coordinates.items() for
                   v, dv in p2coordinates.items()}
time_matrix = {(u, v): np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2) for u, du in p2coordinates.items() for
               v, dv in p2coordinates.items()}

In [12]:


routing_manager = PDRoutingManagerBuilder(
    distance_matrix=distance_matrix,
    time_matrix=time_matrix,
    model_config=ModelConfig(max_execution_time_minutes=1)
)

routing_manager.add_cargos(cargos)
routing_manager.add_tariff(tariff)

routing_manager.add_depo(depo)

# routing_manager.distance_matrix = distance_matrix
# routing_manager.time_matrix = time_matrix


routing_manager = routing_manager.build()

In [13]:
from src.vrp_study.pdptw_model.solution_builder import SolutionBuilder

sol = find_optimal_paths(routing_manager, SolutionBuilder())[0]

2025-07-22 16:43:47.189 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:118 - (1, 32.074999999999996)
2025-07-22 16:43:47.191 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:118 - (1, 16.0875)
2025-07-22 16:43:47.192 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:118 - (1, 8.09375)
2025-07-22 16:43:47.194 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:118 - (1, 4.096875)
2025-07-22 16:43:47.196 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:118 - (1, 2.0984374999999997)
2025-07-22 16:43:47.197 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:118 - (1, 1.09921875)
2025-07-22 16:43:47.198 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:118 - (3, 0.599609375)
2025-07-22 16:43:47.199 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:118 - (3, 0.3498046875)
2025-

  0%|          | 0/1 [00:00<?, ?it/s]

2025-07-22 16:43:47.218 | INFO     | src.vrp_study.pdptw_model.routing_model:find_optimal_paths:443 - problem size: 11
2025-07-22 16:43:47.219 | INFO     | src.vrp_study.pdptw_model.routing_model:do_solve:378 - Начало создания модели
2025-07-22 16:43:47.223 | INFO     | src.vrp_study.pdptw_model.routing_model:add_distance_dimension:299 - Добавление размерности для расстояния
2025-07-22 16:43:47.224 | INFO     | src.vrp_study.pdptw_model.routing_model:add_count_dimension:335 - Добавление размерности для расстояния
2025-07-22 16:43:47.225 | INFO     | src.vrp_study.pdptw_model.routing_model:add_pick_up_and_delivery:269 - Добавление ограничения для порядка доставки
2025-07-22 16:43:47.226 | INFO     | src.vrp_study.pdptw_model.routing_model:add_time_window:194 - add time
2025-07-22 16:43:47.228 | INFO     | src.vrp_study.pdptw_model.routing_model:add_mass_constraint:177 - Добавление ограничений для массы
2025-07-22 16:43:47.236 | INFO     | src.vrp_study.pdptw_model.routing_model:do_solve

  0%|          | 0/1 [00:00<?, ?it/s]

2025-07-22 16:43:47.379 | INFO     | src.vrp_study.pdptw_model.routing_model:find_optimal_paths:443 - problem size: 3
2025-07-22 16:43:47.380 | INFO     | src.vrp_study.pdptw_model.routing_model:do_solve:378 - Начало создания модели
2025-07-22 16:43:47.383 | INFO     | src.vrp_study.pdptw_model.routing_model:add_distance_dimension:299 - Добавление размерности для расстояния
2025-07-22 16:43:47.387 | INFO     | src.vrp_study.pdptw_model.routing_model:add_count_dimension:335 - Добавление размерности для расстояния
2025-07-22 16:43:47.388 | INFO     | src.vrp_study.pdptw_model.routing_model:add_pick_up_and_delivery:269 - Добавление ограничения для порядка доставки
2025-07-22 16:43:47.390 | INFO     | src.vrp_study.pdptw_model.routing_model:add_time_window:194 - add time
2025-07-22 16:43:47.392 | INFO     | src.vrp_study.pdptw_model.routing_model:add_mass_constraint:177 - Добавление ограничений для массы
2025-07-22 16:43:47.397 | INFO     | src.vrp_study.pdptw_model.routing_model:do_solve:

...

2025-07-22 16:43:49.079 | INFO     | src.vrp_study.pdptw_model.routing_model:find_optimal_paths:443 - problem size: 5
2025-07-22 16:43:49.084 | INFO     | src.vrp_study.pdptw_model.routing_model:do_solve:378 - Начало создания модели
2025-07-22 16:43:49.089 | INFO     | src.vrp_study.pdptw_model.routing_model:add_distance_dimension:299 - Добавление размерности для расстояния
2025-07-22 16:43:49.091 | INFO     | src.vrp_study.pdptw_model.routing_model:add_count_dimension:335 - Добавление размерности для расстояния
2025-07-22 16:43:49.093 | INFO     | src.vrp_study.pdptw_model.routing_model:add_pick_up_and_delivery:269 - Добавление ограничения для порядка доставки
2025-07-22 16:43:49.094 | INFO     | src.vrp_study.pdptw_model.routing_model:add_time_window:194 - add time
2025-07-22 16:43:49.097 | INFO     | src.vrp_study.pdptw_model.routing_model:add_mass_constraint:177 - Добавление ограничений для массы
2025-07-22 16:43:49.101 | INFO     | src.vrp_study.pdptw_model.routing_model:do_solve:

  0%|          | 0/1 [00:00<?, ?it/s]

2025-07-22 16:43:49.173 | INFO     | src.vrp_study.pdptw_model.routing_model:find_optimal_paths:443 - problem size: 3
2025-07-22 16:43:49.175 | INFO     | src.vrp_study.pdptw_model.routing_model:do_solve:378 - Начало создания модели
2025-07-22 16:43:49.178 | INFO     | src.vrp_study.pdptw_model.routing_model:add_distance_dimension:299 - Добавление размерности для расстояния
2025-07-22 16:43:49.181 | INFO     | src.vrp_study.pdptw_model.routing_model:add_count_dimension:335 - Добавление размерности для расстояния
2025-07-22 16:43:49.183 | INFO     | src.vrp_study.pdptw_model.routing_model:add_pick_up_and_delivery:269 - Добавление ограничения для порядка доставки
2025-07-22 16:43:49.184 | INFO     | src.vrp_study.pdptw_model.routing_model:add_time_window:194 - add time
2025-07-22 16:43:49.186 | INFO     | src.vrp_study.pdptw_model.routing_model:add_mass_constraint:177 - Добавление ограничений для массы
2025-07-22 16:43:49.191 | INFO     | src.vrp_study.pdptw_model.routing_model:do_solve:

  0%|          | 0/1 [00:00<?, ?it/s]

2025-07-22 16:43:49.261 | INFO     | src.vrp_study.pdptw_model.routing_model:find_optimal_paths:443 - problem size: 3
2025-07-22 16:43:49.263 | INFO     | src.vrp_study.pdptw_model.routing_model:do_solve:378 - Начало создания модели
2025-07-22 16:43:49.266 | INFO     | src.vrp_study.pdptw_model.routing_model:add_distance_dimension:299 - Добавление размерности для расстояния
2025-07-22 16:43:49.268 | INFO     | src.vrp_study.pdptw_model.routing_model:add_count_dimension:335 - Добавление размерности для расстояния
2025-07-22 16:43:49.274 | INFO     | src.vrp_study.pdptw_model.routing_model:add_pick_up_and_delivery:269 - Добавление ограничения для порядка доставки
2025-07-22 16:43:49.276 | INFO     | src.vrp_study.pdptw_model.routing_model:add_time_window:194 - add time
2025-07-22 16:43:49.278 | INFO     | src.vrp_study.pdptw_model.routing_model:add_mass_constraint:177 - Добавление ограничений для массы
2025-07-22 16:43:49.282 | INFO     | src.vrp_study.pdptw_model.routing_model:do_solve:

In [14]:
# sol

# Генерация столбцов

In [15]:
@dataclass
class Route:
    """
        Описание маршрута
    """
    cost: int  # цена
    path: list[int]  # маршрут
    path_set: Optional[set[int]] = None  # множество для блейзинговости

    def __post_init__(self):
        self.path_set = set(self.path)

In [16]:
class SelectModelCpSat:
    """
        модель выбора самых дешевых маршрутов через cp_sat
    """

    def __init__(self, _routes: list[Route], _nodes: set[int]):
        self.routes = _routes  # все маршруты
        self.nodes = _nodes  # все ноды

        self.model = CpModel()
        self.Z = {}
        self._build()

    def _build(self):
        routes = self.routes
        Z = self.Z
        model = self.model

        for i, r in enumerate(routes):
            Z[i] = model.new_bool_var(name=f'Z_{i}')  # брать или не брать маррут
        for n in self.nodes:
            model.add(
                sum(Z[i] for i, route in enumerate(routes) if n in route.path_set) == 1)  # каждая точка посещена 1 раз

    def optimize(self, solver: CpSolver) -> Optional[list[Route]]:
        self.model.clear_objective()
        self.model.minimize(sum(self.Z[i] * r.cost for i, r in enumerate(self.routes)))

        status = solver.solve(self.model)

        log.info(f'model solve with status: {status}')

        if status == CpSolverStatus.OPTIMAL or status == CpSolverStatus.FEASIBLE:
            return [r for i, r in enumerate(self.routes) if solver.Value(self.Z[i]) == 1]
        return None

In [17]:
class PulpSelectModelLP:
    """
       Тоже самое, что выше только в терминах LP для поиска дуальных переменных
    """

    def __init__(self, _routes: list[Route], _nodes: set[int]):
        self.model = pulp.LpProblem(f"MasterProblem_{uuid.uuid4()}", pulp.LpMinimize)
        self.routes = _routes
        self.Z = {}
        self.nodes = _nodes
        self._build()

    def _build(self):
        routes = self.routes
        Z = self.Z
        model = self.model

        for i, r in enumerate(routes):
            Z[i] = pulp.LpVariable(name=f'z_{i}', lowBound=0, upBound=1)

        for n in self.nodes:
            model.addConstraint(
                pulp.lpSum(Z[i] for i, route in enumerate(routes) if n in route.path_set) == 1,
                name=f'cover_{n}'
            )

    def get_dual_variables(self) -> Optional[list[float]]:
        model = self.model
        model.setObjective(pulp.lpSum(self.Z[i] * r.cost for i, r in enumerate(self.routes)))

        model.solve(PULP_CBC_CMD(msg=0))
        # print(model.sol_status)
        if model.sol_status == 1:
            vals = [model.variables()[i].value() for i in range(len(self.routes))]
            log.info(f'master_prob all_int:{all(v in {0.0,0,1.0,1} for v in vals)}')
            duals = {n: model.constraints[f"cover_{n}"].pi for n in self.nodes}
            return duals
        return None

In [18]:
NODES = set(n.id for n in routing_manager.nodes() if n.is_transit)

In [19]:
routes: list[Route] = []
for nodes in routing_manager.get_pick_up_and_delivery_nodes():
    q = [routing_manager.get_depo_index()] + nodes + [routing_manager.get_depo_index()]
    routes.append(
        Route(
            cost=int(
                sum(routing_manager.get_distance(
                    routing_manager.nodes()[q[i]],
                    routing_manager.nodes()[q[i + 1]]
                ) for i in range(len(q) - 1))
            ),
            path=nodes
        )
    )

# for nodes in sol:
#     if len(nodes) == 0:
#         continue
#     routes.append(
#         Route(
#             cost=int(
#                 sum(routing_manager.get_distance(
#                     routing_manager.nodes()[nodes[i]],
#                     routing_manager.nodes()[nodes[i + 1]]
#                 ) for i in range(len(nodes) - 1))
#             ),
#             path=nodes[1:-1]
#         )
#     )

In [20]:
from src.vrp_study.routing_manager import RoutingManager


class PDPTWModel:
    """
        
        Модель pdptw.
        Оставлю это пока без комментариев
        По сути VRP с доп констрейнтами.
    
    """

    def __init__(self, routing_manager: RoutingManager, cost: dict[float], ignore_routes: list[Route]):
        self.routing_manager = routing_manager
        self.cost = cost

        self.model = CpModel()
        self.X = {}
        self.T = {}
        self.Q = {}
        self.ignore_routes = ignore_routes
        self._build()

    def _build(self):
        routing_manager = self.routing_manager
        log.info(f'problem size: {len(routing_manager.nodes())}')

        N = len(routing_manager.nodes())
        M = 1_000_000

        X = self.X
        T = self.T
        Q = self.Q

        model = self.model
        depo = routing_manager.get_depo_index()

        for i in range(N):
            for j in range(N):
                if i == j:
                    X[i, j] = model.new_constant(0)
                else:
                    X[i, j] = model.new_bool_var(f'x_{i, j}')

        max_time = max(n.end_time for n in routing_manager.nodes())
        max_mass = max(car.capacity for car in routing_manager.cars())

        log.debug(f'max_time: {max_time}, max_mass: {max_mass}')
        for i in range(N):
            T[i] = model.new_int_var(lb=0, ub=max_time * 2, name=f'time_{i}')
            Q[i] = model.new_int_var(lb=0, ub=max_mass, name=f'mass_{i}')

        model.add(sum(X[depo, i] for i in range(N)) == 1)

        for i, node in enumerate(routing_manager.nodes()):
            model.add(
                sum(X[i, j] for j in range(N)) == sum(X[j, i] for j in range(N))
            )
            if i == 0:
                continue
            model.add(sum(X[i, j] for j in range(N)) <= 1)
            for j in range(N):
                if i == j:
                    continue
                ni = routing_manager.nodes()[i]
                nj = routing_manager.nodes()[j]

                model.add(
                    T[i] + int(routing_manager.get_time(ni, nj) + routing_manager.nodes()[i].service_time) <=
                    T[j] + M * (1 - X[i, j]))

                model.add(
                    Q[i] + routing_manager.nodes()[i].demand <=
                    Q[j] + M * (1 - X[i, j]))

        for node_a, node_b in routing_manager.get_pick_up_and_delivery_nodes():
            a, b = node_a, node_b
            model.add(T[a] <= T[b])
            model.add(sum(X[k, a] + X[k, b] for k in range(N)) == 2 * sum(X[k, a] for k in range(N)))

        for i, node in enumerate(routing_manager.nodes()):
            model.add(node.start_time <= T[i])
            model.add(T[i] <= node.end_time)

        for route in self.ignore_routes:
            route = route.path
            self.ignore_path(route)

    def ignore_path(self, route: list[int]):
        model = self.model
        X = self.X
        v = X[0, route[0]] + X[route[-1], 0]
        for i in range(len(route) - 1):
            v += X[route[i], route[i + 1]]
        model.add(v != len(route) + 1)

    def optimize(self, solver: CpSolver):
        self.model.clear_objective()
        N = len(self.routing_manager.nodes())
        r = self.routing_manager
        dst = sum(self.X[i, j] * r.get_distance(r.nodes()[i], r.nodes()[j]) for i in range(N) for j in range(N))
        delta = sum(self.cost[r.nodes()[i].id] * sum(self.X[j, i] for j in range(N)) for i in range(1, N))
        self.model.minimize(dst - delta)

        status = solver.solve(self.model)

        log.info(f'model solve with status: {status}')

        if status == CpSolverStatus.OPTIMAL or status == CpSolverStatus.FEASIBLE:
            start = 0
            path = []
            while True:
                next_node = next(iter(n for n in range(N) if solver.Value(self.X[start, n]) == 1))
                if next_node == 0:
                    if path[0] == 0:
                        path = path[1:]
                    return path
                else:
                    start = next_node
                    path.append(start)

        #     return [r for i, r in enumerate(self.routes) if solver.Value(self.Z[i]) == 1]
        # return None

In [21]:

best = {'preferred_variable_order': 2,
        'clause_cleanup_protection': 1,
        'max_presolve_iterations': 5,
        'cp_model_probing_level': 1,
        'presolve_probing_deterministic_time_limit': 10.0,
        'search_branching': 2,
        'feasibility_jump_linearization_level': 0,
        'fp_rounding': 0,
        'polish_lp_solution': True,
        'linearization_level': 0,
        'cut_level': 2,
        'max_all_diff_cut_size': 128,
        'symmetry_level': 0,
        'num_workers': 4}


def get_solver():
    solver = CpSolver()

    for k, v in best.items():
        if isinstance(v, list):
            for ss in v:
                solver.parameters.ignore_subsolvers.append(ss)
        else:
            if 'ignore_subsolvers' in k:
                if v:
                    solver.parameters.ignore_subsolvers.append(k.split(':')[1])
            else:
                exec(f'solver.parameters.{k} = {v}')
    # solver.parameters.use_lns = True
    # solver.parameters.lns_num_threads = 4
    solver.parameters.log_search_progress = False
    solver.parameters.max_time_in_seconds = 60.0 * 10

    # packing_subsolver = sat_parameters_pb2.SatParameters()
    # packing_subsolver.name = "MyPackingSubsolver"
    # packing_subsolver.use_area_energetic_reasoning_in_no_overlap_2d = False
    # packing_subsolver.use_energetic_reasoning_in_no_overlap_2d = False
    # packing_subsolver.use_timetabling_in_no_overlap_2d = False
    # packing_subsolver.max_pairs_pairwise_reasoning_in_no_overlap_2d = 5_000
    # packing_subsolver.
    # # Add the subsolver to the portfolio
    # solver.parameters.subsolver_params.append(packing_subsolver)  # Define the subsolver
    # solver.parameters.extra_subsolvers.append(
    #     packing_subsolver.name
    # )  # Activate the subsolver

    return solver

In [22]:
solver = get_solver()

In [23]:
cp_select_model = SelectModelCpSat(routes, NODES)
sum(r.cost for r in cp_select_model.optimize(solver))

2025-07-22 16:43:52.815 | INFO     | __main__:optimize:31 - model solve with status: 4


3329

In [ ]:
# решаем мастер проблему и тащим из нее дуальные переменные
# в данном контексте под дуальными переменными можно понимать "сколько стоит посетить точку".
pulp_model = PulpSelectModelLP(_routes=routes, _nodes=NODES)
res = pulp_model.get_dual_variables()
pdptw_model = PDPTWModel(routing_manager, cost=res, ignore_routes=routes)

set_paths = set(tuple(r.path) for r in routes)

for _ in trange(30):

    pulp_model = PulpSelectModelLP(_routes=routes, _nodes=NODES)
    res = pulp_model.get_dual_variables()
    pdptw_model.cost = res
    path = pdptw_model.optimize(solver)

    if path is None or solver.objective_value > 0:
        # если ничего не нашли или нет путей, которые могут улучшить маршрут
        break
    # чек на то, что не генерируем то, чот уже ест
    assert tuple(path) not in set_paths
    set_paths.add(tuple(path))
    pdptw_model.ignore_path(path)

    log.info(f'{path}; {solver.objective_value}')

    nodes = [0] + path + [0]
    # добавляем путь
    routes.append(Route(
        cost=int(
            sum(routing_manager.get_distance(
                routing_manager.nodes()[nodes[i]],
                routing_manager.nodes()[nodes[i + 1]]
            ) for i in range(len(nodes) - 1))
        ),
        path=path
    ))
    cp_select_model = SelectModelCpSat(routes, NODES)
    log.info(f'best_score: {sum(r.cost for r in  cp_select_model.optimize(solver))}')

2025-07-22 16:43:52.849 | INFO     | __main__:get_dual_variables:35 - master_prob all_int:True
2025-07-22 16:43:52.850 | INFO     | __main__:_build:26 - problem size: 107
2025-07-22 16:43:53.003 | DEBUG    | __main__:_build:48 - max_time: 1236, max_mass: 200


  0%|          | 0/30 [00:00<?, ?it/s]

2025-07-22 16:43:53.873 | INFO     | __main__:get_dual_variables:35 - master_prob all_int:True


In [ ]:
cp_select_model = SelectModelCpSat(routes, NODES)
log.info(f'best_score: {sum(r.cost for r in  cp_select_model.optimize(solver))}')